# HadesMiMC

In [1]:
import itertools
def compose_permutations(P1, P2, n): # return polytope for F2 \circ F1
    ieqs = []
    eqns = []
    for ieq in P1.inequalities_list():
        ieqs.append(ieq[:2*n+1] + [0]*n + ieq[-1:] + [0])
    for eqn in P1.equations_list():
        eqns.append(eqn[:2*n+1] + [0]*n + eqn[-1:] + [0])
    for ieq in P2.inequalities_list():
        ieqs.append(ieq[:1] + [0]*n + ieq[1:2*n+1] + [0] + ieq[-1:])
    for eqn in P2.equations_list():
        eqns.append(eqn[:1] + [0]*n + eqn[1:2*n+1] + [0] + eqn[-1:])
    M = [[matrix.zero(ZZ, i, j) for j in (n, n, n, 1, 1)] for i in (n, n, 1)]
    M[0][0] = matrix.identity(ZZ, n)
    M[1][2] = matrix.identity(ZZ, n)
    M[2][3] = matrix.identity(ZZ, 1)
    M[2][4] = matrix.identity(ZZ, 1)
    return Polyhedron(ieqs=ieqs, eqns = eqns).linear_transformation(block_matrix(M))

def reduce_to_single_variable(P, n):
    return P.linear_transformation(Matrix(ZZ, [[1]*n + [0]*n + [0], [0]*n + [1]*n + [0], [0]*n + [0]*n + [1]]))

def reduced_parallel_application(Ps, ns):
    return sum(reduce_to_single_variable(P, n) if n > 1 else P for P, n in zip(Ps, ns))

def get_polytope_function(p, n, d): # model could still be improved when d < n-1 as the degree influences the results then
    ieqs = []
    for i in range(2*n):
        ieqs.append([0] + [0]*i + [1] + [0]*(2*n-i))
        ieqs.append([p-1] + [0]*i + [-1] + [0]*(2*n-i))
    ieqs.append([0]*(2*n+1) + [1]) # ordp is at least 0
    ieqs.append([0] + [-1]*n + [d]*n + [d*(p-1)]) # constraints from function
    ieqs.append([-1] + [0]*n + [1]*n + [0]) # special treatment for output character 1
    return Polyhedron(ieqs=ieqs).convex_hull(Polyhedron(vertices=[[0]*(2*n+1)]))
    
def get_polytope_permutation(p, n, d): # model could still be improved when d < n-1 as the degree influences the results then
    ieqs = []
    for i in range(2*n):
        ieqs.append([0] + [0]*i + [1] + [0]*(2*n-i))
        ieqs.append([p-1] + [0]*i + [-1] + [0]*(2*n-i))
    ieqs.append([0]*(2*n+1) + [1]) # ord_p is at least 0
    ieqs.append([0] + [-1]*n + [d]*n + [d*(p-1)]) # constraints from function
    ieqs.append([-1] + [0]*n + [1]*n + [0]) # special treatment for output character 1
    ieqs.append([(p-1)*n-1] + [-1]*n + [0]*n + [0]) # special treatment for input character of maximal wheight
    P1 = Polyhedron(ieqs=ieqs)
    # second polytope for maximal wheight input character
    ieqs = []
    eqns = []
    for i in range(n):
        ieqs.append([0] + [0]*n + [0]*i + [1] + [0]*(n-i))
        ieqs.append([p-1] + [0]*n + [0]*i + [-1] + [0]*(n-i))
        eqns.append([p-1] + [0]*i + [-1] + [0]*(n-1-i) + [0]*(n+1))
    ieqs.append([-1] + [0]*(2*n) + [1]) # ordp is at least 1 in this case
    ieqs.append([-1] + [0]*n + [1]*n + [0]) # special treatment for output character 1
    ieqs.append([(p-1)*n-1] + [0]*n + [-1]*n + [0]) # special treatment for output character of maximal wheight
    P2 = Polyhedron(ieqs=ieqs,eqns=eqns)
    # missing points in third polytope
    P3 = Polyhedron(vertices=[[0]*(2*n+1), [p-1]*2*n + [0]])
    return P1.convex_hull(P2).convex_hull(P3)

def get_polytope_affine_power_permutation(p, d): # returns a polytope for the function (x+a)^d
    _ds = [p-1, d]
    while _ds[-1] != 1:
        _ds.append(_ds[-2] % _ds[-1])
    dinv = pow(d, -1, p-1)
    _costs = [p-1] + [(x*dinv) % (p-1) for x in _ds[1:]]
    ds, costs = _ds[-1:], _costs[-1:]
    for i in range(len(_ds)-2, 0, -1):
        if _ds[i] * costs[-1] > ds[-1]*_costs[i]:
            ds.append(_ds[i])
            costs.append(_costs[i])
    ds = ds[::-1]
    costs = costs[::-1]

    # and now for the curve
    points = [[0, 0, 0]]
    leftover = p-1
    traded = 0
    for d, c in zip(ds, costs):
        x = min(leftover // c, (p-1-traded) // d)
        leftover -= x*c
        traded += x*d
        points.append([traded, p-1-leftover, 0])
    P = Polyhedron(vertices = points)
    Paffine = get_polytope_permutation(p, 1, 1)
    P2 = compose_permutations(Paffine, P, 1)
    return P2

def get_polytope_reduced_linear_permutation(p, n):
    ieqs = []
    for i in range(2):
        ieqs.append([0] + [0]*i + [1] + [0]*(2-i))
        ieqs.append([n*(p-1)] + [0]*i + [-1] + [0]*(2-i))
    ieqs.append([0, 0, 0, 1]) # ord_p is at least 0
    ieqs.append([0, -1, 1, p-1]) # constraints from function
    ieqs.append([-1, 0, 1, 0]) # special treatment for output character 1
    ieqs.append([(p-1)*n-1, -1, 0, 0]) # special treatment for input character of maximal wheight
    P1 = Polyhedron(ieqs=ieqs)
    # second polytope for maximal wheight input character
    ieqs = []
    eqns = []
    ieqs.append([0, 0, 1, 0])
    ieqs.append([n*(p-1), 0, -1, 0])
    eqns.append([p-1, -1, 0, 0])
    ieqs.append([-1, 0, 0, 1]) # ordp is at least 1 in this case
    ieqs.append([-1, 0, 1, 0]) # special treatment for output character 1
    ieqs.append([(p-1)*n-1, 0, -1, 0]) # special treatment for output character of maximal wheight
    P2 = Polyhedron(ieqs=ieqs,eqns=eqns)
    # missing points in third polytope
    P3 = Polyhedron(vertices=[[0]*3, [n*(p-1), n*(p-1), 0]])
    return P1.convex_hull(P2).convex_hull(P3)

In [2]:
def constraint_variables_in_polyhedron(model, P, vrs):
    assert(P.ambient_dimension() == len(vrs))
    vrs = (1, ) + tuple(vrs)
    for ieq in P.inequalities_list():
        model.add_constraint(sum(x*y for x, y in zip(vrs, ieq)) >= 0)
    for eqn in P.equations_list():
        model.add_constraint(sum(x*y for x, y in zip(vrs, eqn)) == 0)

## Generic SPN

### 64-bit prime

In [3]:
# degrees mod p
p = 2**64-2**32+1
d = 7
t = 8

P_cell = get_polytope_affine_power_permutation(p, d)
P_id = Polyhedron(vertices=[[0, 0, 0], [p-1,  p-1, 0]])
P_round = compose_permutations(P_cell + (t-1)*P_id, get_polytope_reduced_linear_permutation(p, t), 1)
P_full = P_round
i = 1
degrees = []
while True:
    model, vrs = P_full.to_linear_program(solver="PPL", return_variable=True)
    model.set_integer(vrs)
    model.set_min(vrs, 0)
    model.set_max(vrs, t*(p-1))
    model.set_objective(vrs[0])
    model.add_constraint(vrs[1] == 1)
    model.add_constraint(vrs[2] <= 0)
    res = model.solve()
    print(i,res)
    degrees.append(res)
    if res >= t*(p-1)-1:
        break
    i += 1
    P_full = compose_permutations(P_full, P_round, 1)

1 7
2 49
3 343
4 2401
5 16807
6 117649
7 823543
8 5764801
9 40353607
10 282475249
11 1977326743
12 13841287201
13 96889010407
14 678223072849
15 4747561509943
16 33232930569601
17 232630513987207
18 1628413597910449
19 11398895185373143
20 79792266297612001
21 558545864083284007
22 3909821048582988049
23 19721315965224060319
24 35532810881865132589
25 51344305798506204859
26 67155800715147277129
27 82967295631788349399
28 98778790548429421669
29 114590285465070493939
30 130401780381711566209
31 146213275298352638479
32 147573952555316674559


In [4]:
# degrees mod p
p = 2**64-2**32+1
d = pow(7, -1, p-1)
t = 8

P_cell = get_polytope_affine_power_permutation(p, d)
P_id = Polyhedron(vertices=[[0, 0, 0], [p-1,  p-1, 0]])
P_round = compose_permutations(P_cell + (t-1)*P_id, get_polytope_reduced_linear_permutation(p, t), 1)
P_full = P_round
i = 1
degrees = []
while True:
    model, vrs = P_full.to_linear_program(solver="PPL", return_variable=True)
    model.set_integer(vrs)
    model.set_min(vrs, 0)
    model.set_max(vrs, t*(p-1))
    model.set_objective(vrs[0])
    model.add_constraint(vrs[1] == 1)
    model.add_constraint(vrs[2] <= 0)
    res = model.solve()
    print(i,res)
    degrees.append(res)
    if res >= t*(p-1)-1:
        break
    i += 1
    P_full = compose_permutations(P_full, P_round, 1)

1 10540996611094048183
2 26352491527735120453
3 42163986444376192723
4 57975481361017264993
5 73786976277658337263
6 89598471194299409533
7 105409966110940481803
8 121221461027581554073
9 137032955944222626343
10 146068095896588953386
11 147358830175498428678
12 147543220786771210862
13 147569562302667322603
14 147573325376366767137
15 147573862958323830642
16 147573939755746268286
17 147573950726806616520
18 147573952294100951982
19 147573952518000142763
20 147573952549985741446
21 147573952554555112686
22 147573952555207880006
23 147573952555301132480
24 147573952555314454262
25 147573952555316357374
26 147573952555316629247
27 147573952555316668086
28 147573952555316673635
29 147573952555316674427
30 147573952555316674541
31 147573952555316674557
32 147573952555316674559


### Degree 2 extension field over 31-bit prime


In [5]:
# q-degrees mod p
p = 2**31-2**24+1
e = 2
d = 7
t = 8

P_cell = get_polytope_affine_power_permutation(p**e, d)
P_id = Polyhedron(vertices=[[0, 0, 0], [p**e-1,  p**e-1, 0]])
P_round = compose_permutations(P_cell + (t-1)*P_id, get_polytope_reduced_linear_permutation(p**e, t), 1)
P_full = P_round
r = 1
qdegrees = []
while True:
    model = MixedIntegerLinearProgram('PPL', maximization=True)
    digits_input_exponents = model.new_variable(integer=True, nonnegative=True)
    model.set_max(digits_input_exponents, t*(p-1))
    rotations_input_exponents = [sum(p**i*digits_input_exponents[(i-j) % e] for i in range(e)) for j in range(e)]
    valuations = model.new_variable(integer=True, nonnegative=True)
    valuation = 0
    for i in range(e):
        valuation += valuations[i]
        constraint_variables_in_polyhedron(model, P_full, [rotations_input_exponents[i], p**i, valuations[i]])
    model.add_constraint(valuation <= 0)
    model.set_objective(rotations_input_exponents[0])
    res = model.solve()
    print(r, res)
    qdegrees.append(res)
    if res >= t*(p**e-1)-1:
        break
    r += 1
    P_full = compose_permutations(P_full, P_round, 1)

1 7
2 49
3 343
4 2401
5 16807
6 117649
7 823543
8 5764801
9 40353607
10 282475249
11 1977326743
12 13841287201
13 96889010407
14 678223072849
15 4747561509943
16 33232930569601
17 232630513987207
18 1628413597910449
19 11398895185373143
20 79792266297612001
21 558545864083284007
22 3909821048582988049
23 7801172394549488179
24 11692523740515988309
25 15583875086482488439
26 19475226432448988569
27 23366577778415488699
28 27257929124381988829
29 31149280470348488959
30 35040631816314989089
31 36319279229020667903


In [6]:
# q-degrees mod p
p = 2**31-2**24+1
e = 2
d = pow(7, -1, p**e-1)
t = 8

P_cell = get_polytope_affine_power_permutation(p**e, d)
P_id = Polyhedron(vertices=[[0, 0, 0], [p**e-1,  p**e-1, 0]])
P_round = compose_permutations(P_cell + (t-1)*P_id, get_polytope_reduced_linear_permutation(p**e, t), 1)
P_full = P_round
r = 1
qdegrees = []
while True:
    model = MixedIntegerLinearProgram('PPL', maximization=True)
    digits_input_exponents = model.new_variable(integer=True, nonnegative=True)
    model.set_max(digits_input_exponents, t*(p-1))
    rotations_input_exponents = [sum(p**i*digits_input_exponents[(i-j) % e] for i in range(e)) for j in range(e)]
    valuations = model.new_variable(integer=True, nonnegative=True)
    valuation = 0
    for i in range(e):
        valuation += valuations[i]
        constraint_variables_in_polyhedron(model, P_full, [rotations_input_exponents[i], p**i, valuations[i]])
    model.add_constraint(valuation <= 0)
    model.set_objective(rotations_input_exponents[0])
    res = model.solve()
    print(r, res)
    qdegrees.append(res)
    if res >= t*(p**e-1)-1:
        break
    r += 1
    P_full = compose_permutations(P_full, P_round, 1)

1 1297117115322166711
2 5188468461288666841
3 9079819807255166971
4 12971171153221667101
5 16862522499188167231
6 20753873845154667361
7 24645225191121167491
8 28536576537087667621
9 32427927883054167751
10 35763371893882596453
11 36239863895429514839
12 36307934181364788894
13 36317658507926970902
14 36319047697435854046
15 36319246153079980210
16 36319274503886283947
17 36319278554001470195
18 36319279132589353945
19 36319279215244765909
20 36319279227052681904
21 36319279228739527046
22 36319279228980504924
23 36319279229014930335
24 36319279229019848251
25 36319279229020550810
26 36319279229020651176
27 36319279229020665514
28 36319279229020667562
29 36319279229020667855
30 36319279229020667897
31 36319279229020667903


In [7]:
# p-degrees mod p
p = 2**31-2**24+1
e = 2
d = 7
t = 8

P_cell = get_polytope_affine_power_permutation(p**e, d)
P_id = Polyhedron(vertices=[[0, 0, 0], [p**e-1,  p**e-1, 0]])
P_round = compose_permutations(P_cell + (t-1)*P_id, get_polytope_reduced_linear_permutation(p**e, t), 1)
P_full = P_round
r = 1
pdegrees = []
while True:
    model = MixedIntegerLinearProgram('PPL', maximization=True)
    digits_input_exponents = model.new_variable(integer=True, nonnegative=True)
    model.set_max(digits_input_exponents, t*(p-1))
    rotations_input_exponents = [sum(p**i*digits_input_exponents[(i-j) % e] for i in range(e)) for j in range(e)]
    valuations = model.new_variable(integer=True, nonnegative=True)
    valuation = 0
    for i in range(e):
        valuation += valuations[i]
        constraint_variables_in_polyhedron(model, P_full, [rotations_input_exponents[i], p**i, valuations[i]])
    model.add_constraint(valuation <= 0)
    model.set_objective(sum(digits_input_exponents[i] for i in range(e)))
    res = model.solve()
    print(r, res)
    pdegrees.append(res)
    if res >= t*e*(p-1)-1:
        break
    r += 1
    P_full = compose_permutations(P_full, P_round, 1)

1 7
2 49
3 343
4 2401
5 16807
6 117649
7 823543
8 5764801
9 40353607
10 282475249
11 1977326743
12 3803646546
13 5629966384
14 7456286456
15 9282608164
16 11108941332
17 12935354714
18 14762329593
19 16593234951
20 17083100185
21 17307792611
22 18880639594
23 20706959394
24 22533279194
25 24359598993
26 26185918793
27 28012238593
28 29838558392
29 31664878192
30 33491197992
31 34091302911


In [8]:
# p-degrees mod p
p = 2**31-2**24+1
e = 2
d = pow(7, -1, p**e-1)
t = 8

P_cell = get_polytope_affine_power_permutation(p**e, d)
P_id = Polyhedron(vertices=[[0, 0, 0], [p**e-1,  p**e-1, 0]])
P_round = compose_permutations(P_cell + (t-1)*P_id, get_polytope_reduced_linear_permutation(p**e, t), 1)
P_full = P_round
r = 1
pdegrees = []
while True:
    model = MixedIntegerLinearProgram('PPL', maximization=True)
    digits_input_exponents = model.new_variable(integer=True, nonnegative=True)
    model.set_max(digits_input_exponents, t*(p-1))
    rotations_input_exponents = [sum(p**i*digits_input_exponents[(i-j) % e] for i in range(e)) for j in range(e)]
    valuations = model.new_variable(integer=True, nonnegative=True)
    valuation = 0
    for i in range(e):
        valuation += valuations[i]
        constraint_variables_in_polyhedron(model, P_full, [rotations_input_exponents[i], p**i, valuations[i]])
    model.add_constraint(valuation <= 0)
    model.set_objective(sum(digits_input_exponents[i] for i in range(e)))
    res = model.solve()
    print(r, res)
    pdegrees.append(res)
    if res >= t*e*(p-1)-1:
        break
    r += 1
    P_full = compose_permutations(P_full, P_round, 1)

1 2435093065
2 6087732663
3 9740372261
4 13393011858
5 17045651456
6 20698291053
7 24350930651
8 28003570249
9 31656209847
10 33743432473
11 34041607134
12 34084203514
13 34090288712
14 34091158025
15 34091282213
16 34091299954
17 34091302489
18 34091302850
19 34091302902
20 34091302910
21 34091302910
22 34091302910
23 34091302910
24 34091302910
25 34091302910
26 34091302910
27 34091302910
28 34091302910
29 34091302910
30 34091302910
31 34091302911


### degree 4 extention field over 17-bit prime

In [9]:
# q-degrees mod p
p = 2**17-1
e = 4
d = 7
t = 8

P_cell = get_polytope_affine_power_permutation(p**e, d)
P_id = Polyhedron(vertices=[[0, 0, 0], [p**e-1,  p**e-1, 0]])
P_round = compose_permutations(P_cell + (t-1)*P_id, get_polytope_reduced_linear_permutation(p**e, t), 1)
P_full = P_round
r = 1
qdegrees = []
while True:
    model = MixedIntegerLinearProgram('PPL', maximization=True)
    digits_input_exponents = model.new_variable(integer=True, nonnegative=True)
    model.set_max(digits_input_exponents, t*(p-1))
    rotations_input_exponents = [sum(p**i*digits_input_exponents[(i-j) % e] for i in range(e)) for j in range(e)]
    valuations = model.new_variable(integer=True, nonnegative=True)
    valuation = 0
    for i in range(e):
        valuation += valuations[i]
        constraint_variables_in_polyhedron(model, P_full, [rotations_input_exponents[i], p**i, valuations[i]])
    model.add_constraint(valuation <= 0)
    model.set_objective(rotations_input_exponents[0])
    res = model.solve()
    print(r, res)
    qdegrees.append(res)
    if res >= t*(p**e-1)-1:
        break
    r += 1
    P_full = compose_permutations(P_full, P_round, 1)

1 7
2 49
3 343
4 2401
5 16807
6 117649
7 823543
8 5764801
9 40353607
10 282475249
11 1977326743
12 13841287201
13 96889010407
14 678223072849
15 4747561509943
16 33232930569601
17 232630513987207
18 1628413597910449
19 11398895185373143
20 79792266297612001
21 558545864083284007
22 3909821048582988049
23 27368747340080916343
24 191581231380566414401
25 444557429737575079267
26 697533628094583744133
27 950509826451592408999
28 1203486024808601073865
29 1456462223165609738731
30 1709438421522618403597
31 1962414619879627068463
32 2215390818236635733329
33 2361111184665414205438
34 2361111184665414205439


In [10]:
# q-degrees mod p
p = 2**17-1
e = 4
d = pow(7, -1, p**e-1)
t = 8

P_cell = get_polytope_affine_power_permutation(p**e, d)
P_id = Polyhedron(vertices=[[0, 0, 0], [p**e-1,  p**e-1, 0]])
P_round = compose_permutations(P_cell + (t-1)*P_id, get_polytope_reduced_linear_permutation(p**e, t), 1)
P_full = P_round
r = 1
qdegrees = []
while True:
    model = MixedIntegerLinearProgram('PPL', maximization=True)
    digits_input_exponents = model.new_variable(integer=True, nonnegative=True)
    model.set_max(digits_input_exponents, t*(p-1))
    rotations_input_exponents = [sum(p**i*digits_input_exponents[(i-j) % e] for i in range(e)) for j in range(e)]
    valuations = model.new_variable(integer=True, nonnegative=True)
    valuation = 0
    for i in range(e):
        valuation += valuations[i]
        constraint_variables_in_polyhedron(model, P_full, [rotations_input_exponents[i], p**i, valuations[i]])
    model.add_constraint(valuation <= 0)
    model.set_objective(rotations_input_exponents[0])
    res = model.solve()
    print(r, res)
    qdegrees.append(res)
    if res >= t*(p**e-1)-1:
        break
    r += 1
    P_full = compose_permutations(P_full, P_round, 1)

1 84325399452336221623
2 337301597809344886489
3 590277796166353551355
4 843253994523362216221
5 1096230192880370881087
6 1349206391237379545953
7 1602182589594388210819
8 1855158787951396875685
9 2108134986308405540551
10 2324971727757270110455
11 2355948405107107906156
12 2360373644728513305542
13 2361005821817285505454
14 2361096132829967248299
15 2361109034403207497277
16 2361110877485098961416
17 2361111140782512027722
18 2361111178396428180051
19 2361111183769844773241
20 2361111184537475715125
21 2361111184647137278252
22 2361111184662803215841
23 2361111184665041206925
24 2361111184665360919937
25 2361111184665406593225
26 2361111184665413117980
27 2361111184665414050088
28 2361111184665414183246
29 2361111184665414202269
30 2361111184665414204987
31 2361111184665414205375
32 2361111184665414205430
33 2361111184665414205438
34 2361111184665414205439


In [11]:
# p^2-degrees mod p
p = 2**17-1
e = 4
d = 7
t = 8

P_cell = get_polytope_affine_power_permutation(p**e, d)
P_id = Polyhedron(vertices=[[0, 0, 0], [p**e-1,  p**e-1, 0]])
P_round = compose_permutations(P_cell + (t-1)*P_id, get_polytope_reduced_linear_permutation(p**e, t), 1)
P_full = P_round
r = 1
p2degrees = []
while True:
    model = MixedIntegerLinearProgram('PPL', maximization=True)
    digits_input_exponents = model.new_variable(integer=True, nonnegative=True)
    model.set_max(digits_input_exponents, t*(p-1))
    rotations_input_exponents = [sum(p**i*digits_input_exponents[(i-j) % e] for i in range(e)) for j in range(e)]
    valuations = model.new_variable(integer=True, nonnegative=True)
    valuation = 0
    for i in range(e):
        valuation += valuations[i]
        constraint_variables_in_polyhedron(model, P_full, [rotations_input_exponents[i], p**i, valuations[i]])
    model.add_constraint(valuation <= 0)
    model.set_objective(sum(sum(p**j*digits_input_exponents[2*i+j] for j in range(2)) for i in range(2)))
    res = model.solve()
    print(r, res)
    p2degrees.append(res)
    if res >= t*2*(p**2-1)-1:
        break
    r += 1
    P_full = compose_permutations(P_full, P_round, 1)

1 7
2 49
3 343
4 2401
5 16807
6 117649
7 823543
8 5764801
9 40353607
10 282475249
11 1977326743
12 13841287201
13 28566664667
14 43292042164
15 58017419864
16 72742798985
17 87468188054
18 102193646764
19 116919592952
20 131648951494
21 137469368456
22 137664441322
23 139029951385
24 148588521825
25 163313899289
26 178039276753
27 192764654216
28 207490031680
29 222215409144
30 236940786608
31 251666164071
32 266391541535
33 274873712639


In [12]:
# p^2-degrees mod p
p = 2**17-1
e = 4
d = pow(7, -1, p**e-1)
t = 8

P_cell = get_polytope_affine_power_permutation(p**e, d)
P_id = Polyhedron(vertices=[[0, 0, 0], [p**e-1,  p**e-1, 0]])
P_round = compose_permutations(P_cell + (t-1)*P_id, get_polytope_reduced_linear_permutation(p**e, t), 1)
P_full = P_round
r = 1
p2degrees = []
while True:
    model = MixedIntegerLinearProgram('PPL', maximization=True)
    digits_input_exponents = model.new_variable(integer=True, nonnegative=True)
    model.set_max(digits_input_exponents, t*(p-1))
    rotations_input_exponents = [sum(p**i*digits_input_exponents[(i-j) % e] for i in range(e)) for j in range(e)]
    valuations = model.new_variable(integer=True, nonnegative=True)
    valuation = 0
    for i in range(e):
        valuation += valuations[i]
        constraint_variables_in_polyhedron(model, P_full, [rotations_input_exponents[i], p**i, valuations[i]])
    model.add_constraint(valuation <= 0)
    model.set_objective(sum(sum(p**j*digits_input_exponents[2*i+j] for j in range(2)) for i in range(2)))
    res = model.solve()
    print(r, res)
    p2degrees.append(res)
    if res >= t*2*(p**2-1)-1:
        break
    r += 1
    P_full = compose_permutations(P_full, P_round, 1)

1 19633836617
2 49084591543
3 78535346469
4 107986101394
5 137436856320
6 166887611245
7 196338366171
8 225789121097
9 255239876023
10 272068878837
11 274473022096
12 274816471133
13 274865535281
14 274872544445
15 274873545754
16 274873688798
17 274873709233
18 274873712153
19 274873712569
20 274873712629
21 274873712637
22 274873712638
23 274873712638
24 274873712638
25 274873712638
26 274873712638
27 274873712638
28 274873712638
29 274873712638
30 274873712638
31 274873712638
32 274873712638
33 274873712639


In [13]:
# p-degrees mod p
p = 2**17-1
e = 4
d = 7
t = 8

P_cell = get_polytope_affine_power_permutation(p**e, d)
P_id = Polyhedron(vertices=[[0, 0, 0], [p**e-1,  p**e-1, 0]])
P_round = compose_permutations(P_cell + (t-1)*P_id, get_polytope_reduced_linear_permutation(p**e, t), 1)
P_full = P_round
r = 1
pdegrees = []
while True:
    model = MixedIntegerLinearProgram('PPL', maximization=True)
    digits_input_exponents = model.new_variable(integer=True, nonnegative=True)
    model.set_max(digits_input_exponents, t*(p-1))
    rotations_input_exponents = [sum(p**i*digits_input_exponents[(i-j) % e] for i in range(e)) for j in range(e)]
    valuations = model.new_variable(integer=True, nonnegative=True)
    valuation = 0
    for i in range(e):
        valuation += valuations[i]
        constraint_variables_in_polyhedron(model, P_full, [rotations_input_exponents[i], p**i, valuations[i]])
    model.add_constraint(valuation <= 0)
    model.set_objective(sum(digits_input_exponents[i] for i in range(e)))
    res = model.solve()
    print(r, res)
    pdegrees.append(res)
    if res >= t*e*(p-1)-1:
        break
    r += 1
    P_full = compose_permutations(P_full, P_round, 1)

1 7
2 49
3 343
4 2401
5 16807
6 117649
7 229999
8 342383
9 454992
10 569185
11 694461
12 897323
13 1122018
14 1346743
15 1491465
16 1605469
17 1729421
18 1923013
19 2147709
20 2372431
21 2529183
22 2643017
23 2765781
24 2951054
25 3175745
26 3400438
27 3567792
28 3680139
29 3792485
30 3904832
31 4017179
32 4129525
33 4194239


In [14]:
# p-degrees mod p
p = 2**17-1
e = 4
d = pow(7, -1, p**e-1)
t = 8

P_cell = get_polytope_affine_power_permutation(p**e, d)
P_id = Polyhedron(vertices=[[0, 0, 0], [p**e-1,  p**e-1, 0]])
P_round = compose_permutations(P_cell + (t-1)*P_id, get_polytope_reduced_linear_permutation(p**e, t), 1)
P_full = P_round
r = 1
pdegrees = []
while True:
    model = MixedIntegerLinearProgram('PPL', maximization=True)
    digits_input_exponents = model.new_variable(integer=True, nonnegative=True)
    model.set_max(digits_input_exponents, t*(p-1))
    rotations_input_exponents = [sum(p**i*digits_input_exponents[(i-j) % e] for i in range(e)) for j in range(e)]
    valuations = model.new_variable(integer=True, nonnegative=True)
    valuation = 0
    for i in range(e):
        valuation += valuations[i]
        constraint_variables_in_polyhedron(model, P_full, [rotations_input_exponents[i], p**i, valuations[i]])
    model.add_constraint(valuation <= 0)
    model.set_objective(sum(digits_input_exponents[i] for i in range(e)))
    res = model.solve()
    print(r, res)
    pdegrees.append(res)
    if res >= t*e*(p-1)-1:
        break
    r += 1
    P_full = compose_permutations(P_full, P_round, 1)

1 374485
2 823868
3 1273251
4 1722633
5 2172016
6 2621399
7 3070783
8 3520165
9 3969548
10 4162140
11 4189652
12 4193582
13 4194145
14 4194224
15 4194236
16 4194236
17 4194236
18 4194236
19 4194236
20 4194236
21 4194238
22 4194238
23 4194238
24 4194238
25 4194238
26 4194238
27 4194238
28 4194238
29 4194238
30 4194238
31 4194238
32 4194238
33 4194239
